In [1]:
#CSV Bibliothek importieren 
import csv
#Pandas importieren und als pd bezeichnen
import pandas as pd
#Numpy importieren und als np bezeichnen 
import numpy as np
#Json Biliothek importieren
import json
#Obere Schranke für die Größe der Daten auf 100,000,000 setzen  
csv.field_size_limit(100000000)
#Maximale Anzahl von Zeilen und Spalten Zeigen 
pd.options.display.max_columns = None
pd.options.display.max_rows = None
#RegEx-Bibliothek importieren importieren
import re
#Import counter
from collections import Counter
#LIWC-Python-Package importieren
import liwc
#Import ast um die Strings zu dictionarys umwandeln zu können
import ast
#Um die Warnings zu Console leiten
import warnings
warnings.filterwarnings('ignore')

---

# LIWC-Kategorien
#### Hier bearbeiten wir nur die Tweets von TweetsKB. Dieselbe Methoden wurden für die Tweets in TweetsCov19 auch benutzt

1. Dataframe(Stichprobe) wird gelesen
2. Die Features TweetIDs, Ist_gelöscht, und Tweetinhalte werden ausgewählt.
3. Zuerst werden LIWC-Kategorien berechenet und als ein  Dictionary gespeicert.
4. Dannach werden die Bewertungen aus der Dictionary extrahiert.
5. Extrahierten Daten werden gespeichert.

---

**TweetsCov19:** *Hier werden TweetsIds, Ist_gelöscht und Tweetinhalte ausgewählt und separat gespeichert* 

In [2]:
#Labels und Pfad für den TweetsCov19
TweetsCov19_labels = ['Tweet_Id', "Benutzername", 'Zeitstempel', 'Abonnenten', 'Abonnierten', 'Retweets', 'Likes', 'Entities', 'Stimmung', 'Erwähnungen', 'Hashtags', 'URLs', 'Text', 'Ist_gelöscht', 'Anzahl der Erwähnungen', 'Anzahl der Hashtags', 'Anzahl der URLs', 'Domäne', 'Textlänge', 'Lesbarkeit']
TweetsCov19_Pfad =  'C:/Mo/tweetDB/tweetscov19/TweetsCov19_tsv/data_sample/tweetscov19_sample/TweetsCov19_stichprobe_lesbarkeit.tsv'

In [4]:
#Stichprobe von TweetsCov19 lesen

df_tweetscov19 = pd.read_csv(TweetsCov19_Pfad, sep='\t', error_bad_lines=False, names=TweetsCov19_labels)
print('Form:', df_tweetscov19.shape)
df_tweetscov19.head()

Form: (9409841, 20)


,Tweet_Id,Benutzername,Zeitstempel,Abonnenten,Abonnierten,Retweets,Likes,Entities,Stimmung,Erwähnungen,Hashtags,URLs,Text,Ist_gelöscht,Anzahl der Erwähnungen,Anzahl der Hashtags,Anzahl der URLs,Domäne,Textlänge,Lesbarkeit
0,1178791637348212736,c57d4fb895c5f4a33d2a2b24709f93b5,2019-09-30 22:00:01+00:00,764191,132,19,180,foodie:Foodie:-2.096663959988218;,1 -1,null;,HalloweenTime?,null;,Has your appetite materialized just as fast as...,True,0,1,0,0,155,76.553
1,1178791641601232896,627f7d4ef584ba33a676a4ebcb861cdb,2019-09-30 22:00:02+00:00,321189,26833,32,43,mainland china:Mainland_China:-1.6252666161640...,1 -1,null;,HongKong,https://www.catholicnewsagency.com/news/amid-h...,The persecution of Catholics in mainland China...,False,0,1,1,catholicnewsagency.com,204,29.468
2,1178791643509612547,7966d5716c1c4f33591b17755184e3d6,2019-09-30 22:00:03+00:00,14209,10623,90,182,brexit:Brexit:-1.582809637810362;hear hear:Hea...,1 -2,null;,Brexit,null;,"Hear, hear..\nEmily: You were an outspoken cri...",False,0,1,0,0,163,79.534
3,1178791644214235137,f17996c036ed4cd981f5448fe00d9ed2,2019-09-30 22:00:03+00:00,2264986,655,60,528,null;,3 -1,amstandardca.,BlueJays,null;,We wrapped up the season with a WINNING week! ...,False,1,1,0,0,132,85.889
4,1178791645371875329,32a54e5f6d8a67de1bffd2e689594dc8,2019-09-30 22:00:03+00:00,171,138,33,117,my family:My_Family:-2.6353832182503094;roll t...,4 -1,null;,null;,null;,Super excited to announce that I will be spend...,False,0,0,0,0,257,61.203


In [5]:
#TweetIDs und Texts werdn gespeichert.
df_tweetscov19 = df_tweetscov19[['Tweet_Id' , 'Ist_gelöscht', 'Text']]

pfad_tsv2 = 'C:/Mo/tweetDB/tweetscov19/TweetsCov19_tsv/data_sample/TweetsCov19_sample/TweetsCov19_stichprobe_IDsText.tsv'
with open(pfad_tsv2, 'w', newline='', encoding="utf-8") as out_file:
        tsv_writer = csv.writer(out_file, delimiter="\t")
        for idx, row in df_tweetscov19.iterrows():
            tsv_writer.writerow(row.values.tolist())

***

**TweetsKB:** *Hier werden TweetsIds, Ist_gelöscht und Tweetinhalte ausgewählt und separat gespeichert* 

In [2]:
#Labels und Pfad für den TweetsKB

TweetsKB_labels    = ['Tweet Id', 'Benutzername', 'Zeitstempel', 'Abonnenten', 'Abonnierten', 'Retweets', 'Likes', 'Text', 'URLs', 'Ist_gelöscht' ]
TweetsKB_Pfad =  'C:/Mo/tweetDB/tweetscov19/TweetsCov19_tsv/data_sample/TweetsKB_stichprobe.tsv'

In [3]:
#TweetsKB wird gelesen

df_tweetskb = pd.read_csv(TweetsKB_Pfad, sep='\t', error_bad_lines=False, names=TweetsKB_labels)
print('Form :', df_tweetskb.shape)
df_tweetskb.head()

Form : (9409841, 10)


,Tweet Id,Benutzername,Zeitstempel,Abonnenten,Abonnierten,Retweets,Likes,Text,URLs,Ist_gelöscht
0,1178791636039589889,DCComics,2019-09-30 22:00:01+00:00,3349606,563,2,13,"Today on #DCDaily, the crew takes a deep dive ...",http://bit.ly/2mlNYpS:-:,False
1,1178791641265639424,seanberdyz,2019-09-30 22:00:02+00:00,1203,499,4,20,𝘪’𝘮 𝘨𝘰𝘯𝘯𝘢 𝘣𝘦 𝘳𝘪𝘨𝘩𝘵 𝘩𝘦𝘳𝘦 ⟶ https://t.co/kOxWQL5...,http://ourownside.carrd.co:-:,True
2,1178791640573579264,AnArtistAtBirth,2019-09-30 22:00:02+00:00,109,215,0,0,❛ 𝘠𝘰𝘶'𝘳𝘦 𝘴𝘪𝘵𝘵𝘪𝘯' 𝘰𝘯 𝘺𝘰𝘶𝘳 𝘧𝘦𝘦𝘭𝘪𝘯𝘨𝘴 . . . 𝘐'𝘮 𝘴𝘪...,null;,True
3,1178791644797255680,lagosboygang,2019-09-30 22:00:03+00:00,571,1401,0,0,@Tiffany_o_ @UG_derrick That's the one i wanna...,null;,True
4,1178791648056217600,Phenom_Hoops,2019-09-30 22:00:04+00:00,25630,1674,1,1,2022 6’9 Perry Smith Jr. showcasing his high-m...,https://www.phenomhoopreport.com/2022-69-perry...,False


In [4]:
#Dataframe wird hier gespeichert.

df_tweetskb= df_tweetskb[['Tweet Id', 'Ist_gelöscht', 'Text']]

TweetsKB_Pfad =  'C:/Mo/tweetDB/tweetscov19/TweetsCov19_tsv/data_sample/TweetsKB_stichprobe_IDsText.tsv'
with open(TweetsKB_Pfad, 'w', newline='', encoding="utf-8") as out_file:
    tsv_writer = csv.writer(out_file, delimiter="\t")
    for idx, row in df_tweetskb.iterrows():
        tsv_writer.writerow(row.values.tolist())

---

In [2]:
#Diese Teil-Code habe ich von dem Github von chbrown/liwc-python bekommen.
# Die Linke is : https://github.com/chbrown/liwc-python
#Hier werden die LIWC-Kategorien für die gegebene Wörter im Text berechnet und und als eine Dictionary zurückgegeben.

def tokenize(text):
    # you may want to use a smarter tokenizer
    for match in re.finditer(r'\w+', text, re.UNICODE):
        yield match.group(0)

parser, Kategorie_namen = liwc.load_token_parser('C:/Mo/tweetDB/tweetscov19/TweetsCov19_tsv/liwc_dictionary/LIWC2015 Dictionary.dic')

def get_liwc(text):
    t_tokens = tokenize(text)
    # now flatmap over all the categories in all of the tokens using a generator:
    t_counts = Counter(Kategorie for token in t_tokens for Kategorie in parser(token))
    # and print the results:
    #print(text_counts)
    d= {}
    d=  dict(t_counts)
    text_len = len(list(tokenize(text)))
    for key in d.keys():
        d[key] = float("{:.2f}".format( (d[key] / text_len) * 100 ))
        #d[key] = (d[key] / text_len) * 100 
    return d

t = 'I love You babe!'
t1 = 'Take care of yourself bro!' 
get_liwc(t1)

{'verb': 20.0,
 'affect': 20.0,
 'posemo': 20.0,
 'focuspresent': 20.0,
 'function': 40.0,
 'prep': 20.0,
 'pronoun': 20.0,
 'ppron': 20.0,
 'you': 20.0,
 'social': 40.0,
 'family': 20.0,
 'male': 20.0,
 'drives': 20.0,
 'affiliation': 20.0,
 'informal': 20.0}

***

**TweetsCov19:** *Hier werden Liwc-Kategories für die Inhalte berechnet und gespeichert* 

In [3]:
TweetsCov19_Pfad = 'C:/Mo/tweetDB/tweetscov19/TweetsCov19_tsv/data_sample/TweetsCov19_sample/TweetsCov19_stichprobe_IDsText.tsv'
df_tweetscov19 = pd.read_csv(TweetsCov19_Pfad, sep='\t', error_bad_lines=False, names=['Tweet Id', 'Ist_gelöscht', 'Text'])
print('Form:', df_tweetscov19.shape)
df_tweetscov19.head()

Form: (9409841, 3)


,Tweet Id,Ist_gelöscht,Text
0,1178791637348212736,True,Has your appetite materialized just as fast as...
1,1178791641601232896,False,The persecution of Catholics in mainland China...
2,1178791643509612547,False,"Hear, hear..\nEmily: You were an outspoken cri..."
3,1178791644214235137,False,We wrapped up the season with a WINNING week! ...
4,1178791645371875329,False,Super excited to announce that I will be spend...


In [5]:
df_tweetscov19['Text'] = df_tweetscov19['Text'].astype(str)
df_tweetscov19['liwc'] = df_tweetscov19.apply(lambda row: get_liwc(row['Text']) , axis=1)

print('Text_Liwc wurde extrahiert!')

Text_Liwc wurde extrahiert!


In [6]:
df_tweetscov19.head()

,Tweet Id,Ist_gelöscht,Text,liwc
0,1178791637348212736,True,Has your appetite materialized just as fast as...,"{'function': 38.46, 'pronoun': 7.69, 'ppron': ..."
1,1178791641601232896,False,The persecution of Catholics in mainland China...,"{'function': 33.33, 'prep': 26.67, 'relativ': ..."
2,1178791643509612547,False,"Hear, hear..\nEmily: You were an outspoken cri...","{'verb': 18.52, 'social': 11.11, 'percept': 3...."
3,1178791644214235137,False,We wrapped up the season with a WINNING week! ...,"{'function': 27.27, 'prep': 13.64, 'drives': 9..."
4,1178791645371875329,False,Super excited to announce that I will be spend...,"{'adj': 8.89, 'affect': 4.44, 'posemo': 4.44, ..."


In [8]:
#TweetIDs und Texts werdn gespeichert.
df_tweetscov19 = df_tweetscov19[['Tweet Id' , 'Ist_gelöscht', 'liwc']]

pfad_tsv2 = 'C:/Mo/tweetDB/tweetscov19/TweetsCov19_tsv/data_sample/TweetsCov19_sample/TweetsCov19_stichprobe_liwc.tsv'
with open(pfad_tsv2, 'w', newline='', encoding="utf-8") as out_file:
        tsv_writer = csv.writer(out_file, delimiter="\t")
        for idx, row in df_tweetscov19.iterrows():
            tsv_writer.writerow(row.values.tolist())

***

**TweetsKB:** *Hier werden Liwc-Kategories für die Inhalte berechnet und gespeichert* 

In [3]:
TweetsKB_Pfad =  'C:/Mo/tweetDB/tweetscov19/TweetsCov19_tsv/data_sample/TweetsKB_stichprobe_IDsText.tsv'
df_tweetskb = pd.read_csv(TweetsKB_Pfad, sep='\t', error_bad_lines=False, names=['Tweet_Id', 'Ist_gelöscht', 'Text'])
print('Form :', df_tweetskb.shape)
df_tweetskb.head()

Form : (9409841, 3)


,Tweet_Id,Ist_gelöscht,Text
0,1178791636039589889,False,"Today on #DCDaily, the crew takes a deep dive ..."
1,1178791641265639424,True,𝘪’𝘮 𝘨𝘰𝘯𝘯𝘢 𝘣𝘦 𝘳𝘪𝘨𝘩𝘵 𝘩𝘦𝘳𝘦 ⟶ https://t.co/kOxWQL5...
2,1178791640573579264,True,❛ 𝘠𝘰𝘶'𝘳𝘦 𝘴𝘪𝘵𝘵𝘪𝘯' 𝘰𝘯 𝘺𝘰𝘶𝘳 𝘧𝘦𝘦𝘭𝘪𝘯𝘨𝘴 . . . 𝘐'𝘮 𝘴𝘪...
3,1178791644797255680,True,@Tiffany_o_ @UG_derrick That's the one i wanna...
4,1178791648056217600,False,2022 6’9 Perry Smith Jr. showcasing his high-m...


In [4]:
df_tweetskb['Text'] = df_tweetskb['Text'].astype(str)
df_tweetskb['liwc'] = df_tweetskb.apply(lambda row: get_liwc(row['Text']) , axis=1)

print('Text_Liwc wurde extrahiert!')

Text_Liwc wurde extrahiert!


In [5]:
df_tweetskb.head()

,Tweet_Id,Ist_gelöscht,Text,liwc
0,1178791636039589889,False,"Today on #DCDaily, the crew takes a deep dive ...","{'function': 28.57, 'prep': 11.43, 'relativ': ..."
1,1178791641265639424,True,𝘪’𝘮 𝘨𝘰𝘯𝘯𝘢 𝘣𝘦 𝘳𝘪𝘨𝘩𝘵 𝘩𝘦𝘳𝘦 ⟶ https://t.co/kOxWQL5...,{}
2,1178791640573579264,True,❛ 𝘠𝘰𝘶'𝘳𝘦 𝘴𝘪𝘵𝘵𝘪𝘯' 𝘰𝘯 𝘺𝘰𝘶𝘳 𝘧𝘦𝘦𝘭𝘪𝘯𝘨𝘴 . . . 𝘐'𝘮 𝘴𝘪...,{}
3,1178791644797255680,True,@Tiffany_o_ @UG_derrick That's the one i wanna...,"{'function': 23.08, 'article': 7.69, 'number':..."
4,1178791648056217600,False,2022 6’9 Perry Smith Jr. showcasing his high-m...,"{'function': 23.08, 'pronoun': 10.26, 'ppron':..."


In [6]:
df_tweetskb = df_tweetskb[['Tweet_Id' , 'Ist_gelöscht', 'liwc']]

TweetsKB_Pfad =  'C:/Mo/tweetDB/tweetscov19/TweetsCov19_tsv/data_sample/TweetsKB_stichprobe_liwc.tsv'

with open(TweetsKB_Pfad, 'w', newline='', encoding="utf-8") as out_file:
    tsv_writer = csv.writer(out_file, delimiter="\t")
    for idx, row in df_tweetskb.iterrows():
        tsv_writer.writerow(row.values.tolist())

***

In [3]:
TweetsCov19_Pfad = 'C:/Mo/tweetDB/tweetscov19/TweetsCov19_tsv/data_sample/TweetsCov19_sample/TweetsCov19_stichprobe_liwc.tsv'
df_tweetscov19 = pd.read_csv(TweetsCov19_Pfad, sep='\t', error_bad_lines=False, names=['Tweet_Id', 'Ist_gelöscht', 'liwc'])
print('Form:', df_tweetscov19.shape)
df_tweetscov19.head()

Form: (9409841, 3)


,Tweet_Id,Ist_gelöscht,liwc
0,1178791637348212736,True,"{'function': 38.46, 'pronoun': 7.69, 'ppron': ..."
1,1178791641601232896,False,"{'function': 33.33, 'prep': 26.67, 'relativ': ..."
2,1178791643509612547,False,"{'verb': 18.52, 'social': 11.11, 'percept': 3...."
3,1178791644214235137,False,"{'function': 27.27, 'prep': 13.64, 'drives': 9..."
4,1178791645371875329,False,"{'adj': 8.89, 'affect': 4.44, 'posemo': 4.44, ..."


In [2]:
TweetsKB_Pfad =  'C:/Mo/tweetDB/tweetscov19/TweetsCov19_tsv/data_sample/TweetsKB_stichprobe_liwc.tsv'
df_tweetskb = pd.read_csv(TweetsKB_Pfad, sep='\t', error_bad_lines=False, names=['Tweet_Id', 'Ist_gelöscht', 'liwc'])
print('Form :', df_tweetskb.shape)
df_tweetskb.head()

Form : (9409841, 3)


,Tweet_Id,Ist_gelöscht,liwc
0,1178791636039589889,False,"{'function': 28.57, 'prep': 11.43, 'relativ': ..."
1,1178791641265639424,True,{}
2,1178791640573579264,True,{}
3,1178791644797255680,True,"{'function': 23.08, 'article': 7.69, 'number':..."
4,1178791648056217600,False,"{'function': 23.08, 'pronoun': 10.26, 'ppron':..."


<br>

- Da die Daten relativ viel sind, werden sie in zwei Teilen bearbeitet und gespeichert.

### Teil 1

In [3]:
df = df_tweetskb
#df = df_tweetscov19

 *Im folgenden sind 37 Methoden geschrieben, die die Ergebnisse für bestimmte Subkategorie lesen und zurückgeben.*
 *Fals kein Ergebnis für eine bestimmte Kategorie gibt, dann wird 0 zurückgegeben.*

In [4]:
#Das Ergebnis für soziale Wörter wird zurückgegeben.
def get_liwc_social(dic):
    d = ast.literal_eval(dic)
    try:
        return d['social']
    except KeyError:
        return 0

In [5]:
#Das Ergebnis für die Subkategorie Familie wird zurückgegeben.
def get_liwc_family(dic):
    d = ast.literal_eval(dic)
    try:
        return d['family']
    except KeyError:
        return 0

In [6]:
#Das Ergebnis für die Subkategorie Freunde wird zurückgegeben.
def get_liwc_friend(dic):
    d = ast.literal_eval(dic)
    try:
        return d['friend']
    except KeyError:
        return 0

In [7]:
#Das Ergebnis für die Subkategorie Weibliche wörter wird zurückgegeben.
def get_liwc_female(dic):
    d = ast.literal_eval(dic)
    try:
        return d['female']
    except KeyError:
        return 0

In [8]:
#Das Ergebnis für die Subkategorie mänliche Wörter wird zurückgegeben.
def get_liwc_male(dic):
    d = ast.literal_eval(dic)
    try:
        return d['male']
    except KeyError:
        return 0


In [9]:
#Das Ergebnis für die Subkategorie Arbeit wird zurückgegeben.
def get_liwc_work(dic):
    d = ast.literal_eval(dic)
    try:
        return d['work']
    except KeyError:
        return 0

In [10]:
#Das Ergebnis für die Subkategorie Freizeit wird zurückgegeben.
def get_liwc_leisure(dic):
    d = ast.literal_eval(dic)
    try:
        return d['leisure']
    except KeyError:
        return 0


In [11]:
#Das Ergebnis für die Subkategorie Zuhause wird zurückgegeben.
def get_liwc_home(dic):
    d = ast.literal_eval(dic)
    try:
        return d['home']
    except KeyError:
        return 0

In [12]:
#Das Ergebnis für die Subkategorie Geld wird zurückgegeben.
def get_liwc_money(dic):
    d = ast.literal_eval(dic)
    try:
        return d['money']
    except KeyError:
        return 0

In [13]:
#Das Ergebnis für die Subkategorie Relegion wird zurückgegeben.
def get_liwc_religion(dic):
    d = ast.literal_eval(dic)
    try:
        return d['relig']
    except KeyError:
        return 0


In [14]:
#Das Ergebnis für die Subkategorie Tod wird zurückgegeben.
def get_liwc_death(dic):
    d = ast.literal_eval(dic)
    try:
        return d['death']
    except KeyError:
        return 0

In [15]:
#Das Ergebnis für die Subkategorie informelle Sprache wird zurückgegeben.
def get_liwc_informal(dic):
    d = ast.literal_eval(dic)
    try:
        return d['informal']
    except KeyError:
        return 0

In [16]:
#Das Ergebnis für die Subkategorie Schimpfwörter wird zurückgegeben.
def get_liwc_swear(dic):
    d = ast.literal_eval(dic)
    try:
        return d['swear']
    except KeyError:
        return 0


In [17]:
#Das Ergebnis für die Subkategorie Netspeak wird zurückgegeben.
def get_liwc_netspeak(dic):
    d = ast.literal_eval(dic)
    try:
        return d['netspeak']
    except KeyError:
        return 0


In [18]:
#Das Ergebnis für die Subkategorie Einnverständis wird zurückgegeben.
def get_liwc_assent(dic):
    d = ast.literal_eval(dic)
    try:
        return d['assent']
    except KeyError:
        return 0

In [19]:
#Das Ergebnis für die Subkategorie nicht fließende Wörter wird zurückgegeben.
def get_liwc_nonfl(dic):
    d = ast.literal_eval(dic)
    try:
        return d['nonflu']
    except KeyError:
        return 0

In [20]:
#Das Ergebnis für die Subkategorie füllwörter wird zurückgegeben.
def get_liwc_filler(dic):
    d = ast.literal_eval(dic)
    try:
        return d['filler']
    except KeyError:
        return 0


In [21]:
#Das Ergebnis für die Subkategorie Körper wird zurückgegeben.
def get_liwc_body(dic):
    d = ast.literal_eval(dic)
    try:
        return d['body']
    except KeyError:
        return 0

In [22]:
#Das Ergebnis für die Subkategorie Gesunheit wird zurückgegeben.
def get_liwc_health(dic):
    d = ast.literal_eval(dic)
    try:
        return d['health']
    except KeyError:
        return 0


In [23]:
#Das Ergebnis für die Subkategorie Sexual wird zurückgegeben.
def get_liwc_sexual(dic):
    d = ast.literal_eval(dic)
    try:
        return d['sexual']
    except KeyError:
        return 0

In [24]:
#Das Ergebnis für die Subkategorie Ingestieren wird zurückgegeben.
def get_liwc_ingest(dic):
    d = ast.literal_eval(dic)
    try:
        return d['ingest']
    except KeyError:
        return 0

In [25]:
#Das Ergebnis für die Subkategorie positive Emotionen wird zurückgegeben.
def get_liwc_posemo(dic):
    d = ast.literal_eval(dic)
    try:
        return d['posemo']
    except KeyError:
        return 0

In [26]:
#Das Ergebnis für die Subkategorie Negative emotionen wird zurückgegeben.
def get_liwc_negemo(dic):
    d = ast.literal_eval(dic)
    try:
        return d['negemo']
    except KeyError:
        return 0


In [27]:
#Das Ergebnis für die Subkategorie Angst wird zurückgegeben.
def get_liwc_anx(dic):
    d = ast.literal_eval(dic)
    try:
        return d['anx']
    except KeyError:
        return 0


In [28]:
#Das Ergebnis für die Subkategorie Zorn wird zurückgegeben.
def get_liwc_anger(dic):
    d = ast.literal_eval(dic)
    try:
        return d['anger']
    except KeyError:
        return 0

In [29]:
#Das Ergebnis für die Subkategorie Traurigkeit wird zurückgegeben.
def get_liwc_sad(dic):
    d = ast.literal_eval(dic)
    try:
        return d['sad']
    except KeyError:
        return 0

In [30]:
#Das Ergebnis für die Subkategorie Einblick wird zurückgegeben.
def get_liwc_insight(dic):
    d = ast.literal_eval(dic)
    try:
        return d['insight']
    except KeyError:
        return 0

In [31]:
#Das Ergebnis für die Subkategorie Ursache wird zurückgegeben.
def get_liwc_cause(dic):
    d = ast.literal_eval(dic)
    try:
        return d['cause']
    except KeyError:
        return 0

In [32]:
#Das Ergebnis für die Subkategorie Unstimmigkeit wird zurückgegeben.
def get_liwc_discrep(dic):
    d = ast.literal_eval(dic)
    try:
        return d['discrep']
    except KeyError:
        return 0

In [33]:
#Das Ergebnis für die Subkategorie Unsicherheit wird zurückgegeben.
def get_liwc_tentat(dic):
    d = ast.literal_eval(dic)
    try:
        return d['tentat']
    except KeyError:
        return 0

In [34]:
#Das Ergebnis für die Subkategorie Sicherheit wird zurückgegeben.
def get_liwc_certain(dic):
    d = ast.literal_eval(dic)
    try:
        return d['certain']
    except KeyError:
        return 0

In [35]:
#Das Ergebnis für die Subkategorie Abgerenzung wird zurückgegeben.
def get_liwc_differ(dic):
    d = ast.literal_eval(dic)
    try:
        return d['differ']
    except KeyError:
        return 0

In [36]:
#Das Ergebnis für die Subkategorie Zugehörigkeit wird zurückgegeben.
def get_liwc_affiliation(dic):
    d = ast.literal_eval(dic)
    try:
        return d['affiliation']
    except KeyError:
        return 0

In [37]:
#Das Ergebnis für die Subkategorie Erreichen wird zurückgegeben.
def get_liwc_achiev(dic):
    d = ast.literal_eval(dic)
    try:
        return d['achiev']
    except KeyError:
        return 0

In [38]:
#Das Ergebnis für die Subkategorie Power wird zurückgegeben.
def get_liwc_power(dic):
    d = ast.literal_eval(dic)
    try:
        return d['power']
    except KeyError:
        return 0

In [39]:
#Das Ergebnis für die Subkategorie Belohnung wird zurückgegeben.
def get_liwc_reward(dic):
    d = ast.literal_eval(dic)
    try:
        return d['reward']
    except KeyError:
        return 0


In [40]:
#Das Ergebnis für die Subkategorie Risiko wird zurückgegeben.
def get_liwc_risk(dic):
    d = ast.literal_eval(dic)
    try:
        return d['risk']
    except KeyError:
        return 0

---

In [41]:
#Für alle Tweets die Methode get_liwc_social() wird gerufen. Die Ausgabe der Methode wird gespeichert. 
df['social'] = df.apply(lambda row: get_liwc_social(row['liwc']), axis=1)
print('Liwc_social wurde extrahiert!')   

Liwc_social wurde extrahiert!


In [42]:
#Für alle Tweets die Methode get_liwc_family() wird gerufen. Die Ausgabe der Methode wird gespeichert. 
df['family'] = df.apply(lambda row: get_liwc_family(row['liwc']), axis=1)
print('Liwc_family wurde extrahiert!')

Liwc_family wurde extrahiert!


In [43]:
#Für alle Tweets die Methode get_liwc_friend() wird gerufen. Die Ausgabe der Methode wird gespeichert. 
df['friend'] = df.apply(lambda row: get_liwc_friend(row['liwc']), axis=1)
print('Liwc_friend wurde extrahiert!')

Liwc_friend wurde extrahiert!


In [44]:
#Für alle Tweets die Methode get_liwc_female() wird gerufen. Die Ausgabe der Methode wird gespeichert. 
df['female'] = df.apply(lambda row: get_liwc_female(row['liwc']), axis=1)
print('Liwc_female wurde extrahiert!')

Liwc_female wurde extrahiert!


In [45]:
#Für alle Tweets die Methode get_liwc_male() wird gerufen. Die Ausgabe der Methode wird gespeichert. 
df['male'] = df.apply(lambda row: get_liwc_male(row['liwc']), axis=1)
print('Liwc_male wurde extrahiert!')

Liwc_male wurde extrahiert!


In [46]:
#Für alle Tweets die Methode get_liwc_work() wird gerufen. Die Ausgabe der Methode wird gespeichert. 
df['work'] = df.apply(lambda row: get_liwc_work(row['liwc']), axis=1)
print('Liwc_work wurde extrahiert!')

Liwc_work wurde extrahiert!


In [47]:
#Für alle Tweets die Methode get_liwc_leisure() wird gerufen. Die Ausgabe der Methode wird gespeichert. 
df['leisure'] = df.apply(lambda row: get_liwc_leisure(row['liwc']), axis=1)
print('Liwc_leisure wurde extrahiert!')

Liwc_leisure wurde extrahiert!


In [48]:
#Für alle Tweets die Methode get_liwc_home() wird gerufen. Die Ausgabe der Methode wird gespeichert. 
df['home'] = df.apply(lambda row: get_liwc_home(row['liwc']), axis=1)
print('Liwc_home wurde extrahiert!')

Liwc_home wurde extrahiert!


In [49]:
#Für alle Tweets die Methode get_liwc_money() wird gerufen. Die Ausgabe der Methode wird gespeichert. 
df['money'] = df.apply(lambda row: get_liwc_money(row['liwc']), axis=1)
print('Liwc_home wurde extrahiert!')

Liwc_home wurde extrahiert!


In [50]:
#Für alle Tweets die Methode get_liwc_religion() wird gerufen. Die Ausgabe der Methode wird gespeichert. 
df['religion'] = df.apply(lambda row: get_liwc_religion(row['liwc']), axis=1)
print('Liwc_religion wurde extrahiert!')

Liwc_religion wurde extrahiert!


In [51]:
#Für alle Tweets die Methode get_liwc_death() wird gerufen. Die Ausgabe der Methode wird gespeichert. 
df['death'] = df.apply(lambda row: get_liwc_death(row['liwc']), axis=1)
print('Liwc_death wurde extrahiert!')

Liwc_death wurde extrahiert!


In [52]:
#Für alle Tweets die Methode get_liwc_informal() wird gerufen. Die Ausgabe der Methode wird gespeichert. 
df['informal'] = df.apply(lambda row: get_liwc_informal(row['liwc']), axis=1)
print('Liwc_informal wurde extrahiert!')

Liwc_informal wurde extrahiert!


In [53]:
#Für alle Tweets die Methode get_liwc_swear() wird gerufen. Die Ausgabe der Methode wird gespeichert. 
df['swear'] = df.apply(lambda row: get_liwc_swear(row['liwc']), axis=1)
print('Liwc_swear wurde extrahiert!')

Liwc_swear wurde extrahiert!


In [54]:
#Für alle Tweets die Methode get_liwc_netspeak() wird gerufen. Die Ausgabe der Methode wird gespeichert. 
df['netspeak'] = df.apply(lambda row: get_liwc_netspeak(row['liwc']), axis=1)
print('Liwc_netspeak wurde extrahiert!')

Liwc_netspeak wurde extrahiert!


In [55]:
#Für alle Tweets die Methode get_liwc_assent() wird gerufen. Die Ausgabe der Methode wird gespeichert. 
df['assent'] = df.apply(lambda row: get_liwc_assent(row['liwc']), axis=1)
print('Liwc_assent wurde extrahiert!')

Liwc_assent wurde extrahiert!


In [56]:
#Für alle Tweets die Methode get_liwc_nonfl() wird gerufen. Die Ausgabe der Methode wird gespeichert. 
df['nonfl'] = df.apply(lambda row: get_liwc_nonfl(row['liwc']), axis=1)
print('Liwc_nonfl wurde extrahiert!')


Liwc_nonfl wurde extrahiert!


In [57]:
#Für alle Tweets die Methode get_liwc_filler() wird gerufen. Die Ausgabe der Methode wird gespeichert. 
df['filler'] = df.apply(lambda row: get_liwc_filler(row['liwc']), axis=1)
print('Liwc_filler wurde extrahiert!')

Liwc_filler wurde extrahiert!


In [58]:
#Für alle Tweets die Methode get_liwc_body() wird gerufen. Die Ausgabe der Methode wird gespeichert. 
df['body'] = df.apply(lambda row: get_liwc_body(row['liwc']), axis=1)
print('Liwc_body wurde extrahiert!')

Liwc_body wurde extrahiert!


In [59]:
#Für alle Tweets die Methode get_liwc_health() wird gerufen. Die Ausgabe der Methode wird gespeichert. 
df['health'] = df.apply(lambda row: get_liwc_health(row['liwc']), axis=1)
print('Liwc_health wurde extrahiert!')

Liwc_health wurde extrahiert!


In [60]:
#Für alle Tweets die Methode get_liwc_sexual() wird gerufen. Die Ausgabe der Methode wird gespeichert. 
df['sexual'] = df.apply(lambda row: get_liwc_sexual(row['liwc']), axis=1)
print('Liwc_sexual wurde extrahiert!')

Liwc_sexual wurde extrahiert!


In [61]:
#Für alle Tweets die Methode get_liwc_ingest() wird gerufen. Die Ausgabe der Methode wird gespeichert. 
df['ingest'] = df.apply(lambda row: get_liwc_ingest(row['liwc']), axis=1)
print('Liwc_ingest wurde extrahiert!')

Liwc_ingest wurde extrahiert!


In [62]:
#Für alle Tweets die Methode get_liwc_posemo() wird gerufen. Die Ausgabe der Methode wird gespeichert. 
df['posemo'] = df.apply(lambda row: get_liwc_posemo(row['liwc']), axis=1)
print('Liwc_posemo wurde extrahiert!')

Liwc_posemo wurde extrahiert!


In [63]:
#Für alle Tweets die Methode get_liwc_negemo() wird gerufen. Die Ausgabe der Methode wird gespeichert. 
df['negemo'] = df.apply(lambda row: get_liwc_negemo(row['liwc']), axis=1)
print('Liwc_negemo wurde extrahiert!')

Liwc_negemo wurde extrahiert!


In [64]:
#Für alle Tweets die Methode get_liwc_anx() wird gerufen. Die Ausgabe der Methode wird gespeichert. 
df['anx'] = df.apply(lambda row: get_liwc_anx(row['liwc']), axis=1)
print('Liwc_anx wurde extrahiert!')

Liwc_anx wurde extrahiert!


In [65]:
#Für alle Tweets die Methode get_liwc_anger() wird gerufen. Die Ausgabe der Methode wird gespeichert. 
df['anger'] = df.apply(lambda row: get_liwc_anger(row['liwc']), axis=1)
print('Liwc_anger wurde extrahiert!')

Liwc_anger wurde extrahiert!


In [66]:
#Für alle Tweets die Methode get_liwc_sad() wird gerufen. Die Ausgabe der Methode wird gespeichert. 
df['sad'] = df.apply(lambda row: get_liwc_sad(row['liwc']), axis=1)
print('Liwc_sad wurde extrahiert!')

Liwc_sad wurde extrahiert!


In [67]:
#Für alle Tweets die Methode get_liwc_insight() wird gerufen. Die Ausgabe der Methode wird gespeichert. 
df['insight'] = df.apply(lambda row: get_liwc_insight(row['liwc']), axis=1)
print('Liwc_insight wurde extrahiert!')

Liwc_insight wurde extrahiert!


In [68]:
#Für alle Tweets die Methode get_liwc_cause() wird gerufen. Die Ausgabe der Methode wird gespeichert. 
df['cause'] = df.apply(lambda row: get_liwc_cause(row['liwc']), axis=1)
print('Liwc_cause wurde extrahiert!')

Liwc_cause wurde extrahiert!


In [69]:
#Für alle Tweets die Methode get_liwc_discrep() wird gerufen. Die Ausgabe der Methode wird gespeichert. 
df['discrep'] = df.apply(lambda row: get_liwc_discrep(row['liwc']), axis=1)
print('Liwc_discrep wurde extrahiert!')

Liwc_discrep wurde extrahiert!


In [70]:
#Für alle Tweets die Methode get_liwc_tentat() wird gerufen. Die Ausgabe der Methode wird gespeichert. 
df['tentat'] = df.apply(lambda row: get_liwc_tentat(row['liwc']), axis=1)
print('Liwc_tentat wurde extrahiert!')

Liwc_tentat wurde extrahiert!


In [71]:
#Für alle Tweets die Methode get_liwc_certain() wird gerufen. Die Ausgabe der Methode wird gespeichert. 
df['certain'] = df.apply(lambda row: get_liwc_certain(row['liwc']), axis=1)
print('Liwc_certain wurde extrahiert!')

Liwc_certain wurde extrahiert!


In [72]:
#Für alle Tweets die Methode get_liwc_differ() wird gerufen. Die Ausgabe der Methode wird gespeichert. 
df['differ'] = df.apply(lambda row: get_liwc_differ(row['liwc']), axis=1)
print('Liwc_differ wurde extrahiert!')

Liwc_differ wurde extrahiert!


In [73]:
#Für alle Tweets die Methode get_liwc_affiliation() wird gerufen. Die Ausgabe der Methode wird gespeichert. 
df['affiliation'] = df.apply(lambda row: get_liwc_affiliation(row['liwc']), axis=1)
print('Liwc_affiliation wurde extrahiert!')

Liwc_affiliation wurde extrahiert!


In [74]:
#Für alle Tweets die Methode get_liwc_achiev() wird gerufen. Die Ausgabe der Methode wird gespeichert. 
df['achiev'] = df.apply(lambda row: get_liwc_achiev(row['liwc']), axis=1)
print('Liwc_achiev wurde extrahiert!')

Liwc_achiev wurde extrahiert!


In [75]:
#Für alle Tweets die Methode get_liwc_power() wird gerufen. Die Ausgabe der Methode wird gespeichert. 
df['power'] = df.apply(lambda row: get_liwc_power(row['liwc']), axis=1)
print('Liwc_power wurde extrahiert!')

Liwc_power wurde extrahiert!


In [76]:
#Für alle Tweets die Methode get_liwc_power() wird gerufen. Die Ausgabe der Methode wird gespeichert. 
df['reward'] = df.apply(lambda row: get_liwc_power(row['liwc']), axis=1)
print('Liwc_reward wurde extrahiert!')

Liwc_reward wurde extrahiert!


In [77]:
#Für alle Tweets die Methode get_liwc_risk() wird gerufen. Die Ausgabe der Methode wird gespeichert. 
df['risk'] = df.apply(lambda row: get_liwc_risk(row['liwc']), axis=1)
print('Liwc_risk wurde extrahiert!')

Liwc_risk wurde extrahiert!


In [78]:
#Die Ergebnisse Speichern.
TweetsKB_Pfad =  'C:/Mo/tweetDB/tweetscov19/TweetsCov19_tsv/data_sample/TweetsKB_stichprobe_liwc_risk.tsv'

with open(TweetsKB_Pfad, 'w', newline='', encoding="utf-8") as out_file:
    tsv_writer = csv.writer(out_file, delimiter="\t")
    for idx, row in df.iterrows():
        tsv_writer.writerow(row.values.tolist())

----

In [3]:
TweetsCov19_Pfad = 'C:/Mo/tweetDB/tweetscov19/TweetsCov19_tsv/data_sample/TweetsCov19_sample/TweetsCov19_stichprobe_liwc.tsv'
df_tweetscov19 = pd.read_csv(TweetsCov19_Pfad, sep='\t', error_bad_lines=False, names=['Tweet_Id', 'Ist_gelöscht', 'liwc'])
print('Form:', df_tweetscov19.shape)
df_tweetscov19.head()

Form: (9409841, 3)


,Tweet_Id,Ist_gelöscht,liwc
0,1178791637348212736,True,"{'function': 38.46, 'pronoun': 7.69, 'ppron': ..."
1,1178791641601232896,False,"{'function': 33.33, 'prep': 26.67, 'relativ': ..."
2,1178791643509612547,False,"{'verb': 18.52, 'social': 11.11, 'percept': 3...."
3,1178791644214235137,False,"{'function': 27.27, 'prep': 13.64, 'drives': 9..."
4,1178791645371875329,False,"{'adj': 8.89, 'affect': 4.44, 'posemo': 4.44, ..."


***

In [2]:
TweetsKB_Pfad =  'C:/Mo/tweetDB/tweetscov19/TweetsCov19_tsv/data_sample/TweetsKB_stichprobe_liwc.tsv'
df_tweetskb = pd.read_csv(TweetsKB_Pfad, sep='\t', error_bad_lines=False, names=['Tweet_Id', 'Ist_gelöscht', 'liwc'])
print('Form :', df_tweetskb.shape)
df_tweetskb.head()

Form : (9409841, 3)


,Tweet_Id,Ist_gelöscht,liwc
0,1178791636039589889,False,"{'function': 28.57, 'prep': 11.43, 'relativ': ..."
1,1178791641265639424,True,{}
2,1178791640573579264,True,{}
3,1178791644797255680,True,"{'function': 23.08, 'article': 7.69, 'number':..."
4,1178791648056217600,False,"{'function': 23.08, 'pronoun': 10.26, 'ppron':..."


***

### Teil 2

In [5]:
df = df_tweetskb
#df = df_tweetscov19

In [6]:
#Das Ergebnis für die Subkategorie Fokus auf die Vergangenheit wird zurückgegeben.
def get_liwc_focuspast(dic):
    d = ast.literal_eval(dic)
    try:
        return d['focuspast']
    except KeyError:
        return 0

In [7]:
#Das Ergebnis für die Subkategorie Fokus auf die Peresent wird zurückgegeben.
def get_liwc_focuspresent(dic):
    d = ast.literal_eval(dic)
    try:
        return d['focuspresent']
    except KeyError:
        return 0


In [8]:
#Das Ergebnis für die Subkategorie Fokus auf die Zukunft wird zurückgegeben.
def get_liwc_focusfuture(dic):
    d = ast.literal_eval(dic)
    try:
        return d['focusfuture']
    except KeyError:
        return 0


In [9]:
#Das Ergebnis für die Subkategorie Ich wird zurückgegeben.
def get_liwc_i(dic):
    d = ast.literal_eval(dic)
    try:
        return d['i']
    except KeyError:
        return 0

In [10]:
#Das Ergebnis für die Subkategorie We wird zurückgegeben.
def get_liwc_we(dic):
    d = ast.literal_eval(dic)
    try:
        return d['we']
    except KeyError:
        return 0

In [11]:
#Das Ergebnis für die Subkategorie You wird zurückgegeben.
def get_liwc_you(dic):
    d = ast.literal_eval(dic)
    try:
        return d['you']
    except KeyError:
        return 0

In [12]:
#Das Ergebnis für die Subkategorie Schehe wird zurückgegeben.
def get_liwc_shehe(dic):
    d = ast.literal_eval(dic)
    try:
        return d['shehe']
    except KeyError:
        return 0

In [13]:
#Das Ergebnis für die Subkategorie They wird zurückgegeben.
def get_liwc_they(dic):
    d = ast.literal_eval(dic)
    try:
        return d['they']
    except KeyError:
        return 0

In [14]:
#Das Ergebnis für die Subkategorie IPron wird zurückgegeben.
def get_liwc_ipron(dic):
    d = ast.literal_eval(dic)
    try:
        return d['ipron']
    except KeyError:
        return 0

In [15]:
#Das Ergebnis für die Subkategorie die Artikel wird zurückgegeben.
def get_liwc_article(dic):
    d = ast.literal_eval(dic)
    try:
        return d['article']
    except KeyError:
        return 0

In [16]:
#Das Ergebnis für die Subkategorie Preposition wird zurückgegeben.
def get_liwc_prep(dic):
    d = ast.literal_eval(dic)
    try:
        return d['prep']
    except KeyError:
        return 0

In [17]:
#Das Ergebnis für die Subkategorie Hilfsverb wird zurückgegeben.
def get_liwc_auxverb(dic):
    d = ast.literal_eval(dic)
    try:
        return d['auxverb']
    except KeyError:
        return 0

In [18]:
#Das Ergebnis für die Subkategorie Adverb wird zurückgegeben.
def get_liwc_adverb(dic):
    d = ast.literal_eval(dic)
    try:
        return d['adverb']
    except KeyError:
        return 0

In [19]:
#Das Ergebnis für die Subkategorie Familie Konijugation wird zurückgegeben.
def get_liwc_conj(dic):
    d = ast.literal_eval(dic)
    try:
        return d['conj']
    except KeyError:
        return 0

In [20]:
#Das Ergebnis für die Subkategorie Neation wird zurückgegeben.
def get_liwc_negate(dic):
    d = ast.literal_eval(dic)
    try:
        return d['negate']
    except KeyError:
        return 0


In [21]:
#Das Ergebnis für die Subkategorie Verb wird zurückgegeben.
def get_liwc_verb(dic):
    d = ast.literal_eval(dic)
    try:
        return d['verb']
    except KeyError:
        return 0

In [22]:
#Das Ergebnis für die Subkategorie Adjectiv wird zurückgegeben.
def get_liwc_adj(dic):
    d = ast.literal_eval(dic)
    try:
        return d['adj']
    except KeyError:
        return 0

In [23]:
#Das Ergebnis für die Subkategorie Komparativ wird zurückgegeben.
def get_liwc_compare(dic):
    d = ast.literal_eval(dic)
    try:
        return d['compare']
    except KeyError:
        return 0

In [24]:
#Das Ergebnis für die Subkategorie Interogativ wird zurückgegeben.
def get_liwc_interrog(dic):
    d = ast.literal_eval(dic)
    try:
        return d['interrog']
    except KeyError:
        return 0

In [25]:
#Das Ergebnis für die Subkategorie Zahlen wird zurückgegeben.
def get_liwc_number(dic):
    d = ast.literal_eval(dic)
    try:
        return d['number']
    except KeyError:
        return 0

In [26]:
#Das Ergebnis für die Subkategorie Quantitaiv wird zurückgegeben.
def get_liwc_quant(dic):
    d = ast.literal_eval(dic)
    try:
        return d['quant']
    except KeyError:
        return 0

In [27]:
#Das Ergebnis für die Subkategorie Sehen wird zurückgegeben.
def get_liwc_see(dic):
    d = ast.literal_eval(dic)
    try:
        return d['see']
    except KeyError:
        return 0

In [28]:
#Das Ergebnis für die Subkategorie Hören wird zurückgegeben.
def get_liwc_hear(dic):
    d = ast.literal_eval(dic)
    try:
        return d['hear']
    except KeyError:
        return 0

In [29]:
#Das Ergebnis für die Subkategorie Fühlen wird zurückgegeben.
def get_liwc_feel(dic):
    d = ast.literal_eval(dic)
    try:
        return d['feel']
    except KeyError:
        return 0

In [30]:
#Das Ergebnis für die Subkategorie Emotion wird zurückgegeben.
def get_liwc_motion(dic):
    d = ast.literal_eval(dic)
    try:
        return d['motion']
    except KeyError:
        return 0

In [31]:
#Das Ergebnis für die Subkategorie Raum wird zurückgegeben.
def get_liwc_space(dic):
    d = ast.literal_eval(dic)
    try:
        return d['space']
    except KeyError:
        return 0

In [32]:
#Das Ergebnis für die Subkategorie Zeit wird zurückgegeben.
def get_liwc_time(dic):
    d = ast.literal_eval(dic)
    try:
        return d['time']
    except KeyError:
        return 0


In [33]:
#Für alle Tweets die Methode get_liwc_focuspast() wird gerufen. Die Ausgabe der Methode wird gespeichert. 
df['focuspast'] = df.apply(lambda row: get_liwc_focuspast(row['liwc']), axis=1)
print('Liwc_focuspast wurde extrahiert!')

Liwc_focuspast wurde extrahiert!


In [34]:
#Für alle Tweets die Methode get_liwc_focuspresent() wird gerufen. Die Ausgabe der Methode wird gespeichert. 
df['focuspresent'] = df.apply(lambda row: get_liwc_focuspresent(row['liwc']), axis=1)
print('Liwc_focuspresent wurde extrahiert!')

Liwc_focuspresent wurde extrahiert!


In [35]:
#Für alle Tweets die Methode get_liwc_focusfuture() wird gerufen. Die Ausgabe der Methode wird gespeichert. 
df['focusfuture'] = df.apply(lambda row: get_liwc_focusfuture(row['liwc']), axis=1)
print('Liwc_focusfuture wurde extrahiert!')

Liwc_focusfuture wurde extrahiert!


In [36]:
#Für alle Tweets die Methode get_liwc_i() wird gerufen. Die Ausgabe der Methode wird gespeichert. 
df['i'] = df.apply(lambda row: get_liwc_i(row['liwc']), axis=1)
print('Liwc_i wurde extrahiert!')

Liwc_i wurde extrahiert!


In [37]:
#Für alle Tweets die Methode get_liwc_we() wird gerufen. Die Ausgabe der Methode wird gespeichert. 
df['we'] = df.apply(lambda row: get_liwc_we(row['liwc']), axis=1)
print('Liwc_we wurde extrahiert!')

Liwc_we wurde extrahiert!


In [38]:
#Für alle Tweets die Methode get_liwc_you() wird gerufen. Die Ausgabe der Methode wird gespeichert. 
df['you'] = df.apply(lambda row: get_liwc_you(row['liwc']), axis=1)
print('Liwc_you wurde extrahiert!')

Liwc_you wurde extrahiert!


In [39]:
#Für alle Tweets die Methode get_liwc_shehe() wird gerufen. Die Ausgabe der Methode wird gespeichert. 
df['shehe'] = df.apply(lambda row: get_liwc_shehe(row['liwc']), axis=1)
print('Liwc_shehe wurde extrahiert!')

Liwc_shehe wurde extrahiert!


In [40]:
#Für alle Tweets die Methode get_liwc_they() wird gerufen. Die Ausgabe der Methode wird gespeichert. 
df['they'] = df.apply(lambda row: get_liwc_they(row['liwc']), axis=1)
print('Liwc_they wurde extrahiert!')

Liwc_they wurde extrahiert!


In [41]:
#Für alle Tweets die Methode get_liwc_ipron() wird gerufen. Die Ausgabe der Methode wird gespeichert. 
df['ipron'] = df.apply(lambda row: get_liwc_ipron(row['liwc']), axis=1)
print('Liwc_ipron wurde extrahiert!')

Liwc_ipron wurde extrahiert!


In [42]:
#Für alle Tweets die Methode get_liwc_article() wird gerufen. Die Ausgabe der Methode wird gespeichert. 
df['article'] = df.apply(lambda row: get_liwc_article(row['liwc']), axis=1)
print('Liwc_article wurde extrahiert!')

Liwc_article wurde extrahiert!


In [43]:
#Für alle Tweets die Methode get_liwc_prep() wird gerufen. Die Ausgabe der Methode wird gespeichert. 
df['prep'] = df.apply(lambda row: get_liwc_prep(row['liwc']), axis=1)
print('Liwc_prep wurde extrahiert!')

Liwc_prep wurde extrahiert!


In [44]:
#Für alle Tweets die Methode get_liwc_auxverb() wird gerufen. Die Ausgabe der Methode wird gespeichert. 
df['auxverb'] = df.apply(lambda row: get_liwc_auxverb(row['liwc']), axis=1)
print('Liwc_auxverb wurde extrahiert!')

Liwc_auxverb wurde extrahiert!


In [45]:
#Für alle Tweets die Methode get_liwc_adverb() wird gerufen. Die Ausgabe der Methode wird gespeichert. 
df['adverb'] = df.apply(lambda row: get_liwc_adverb(row['liwc']), axis=1)
print('Liwc_adverb wurde extrahiert!')

Liwc_adverb wurde extrahiert!


In [46]:
#Für alle Tweets die Methode get_liwc_conj() wird gerufen. Die Ausgabe der Methode wird gespeichert. 
df['conj'] = df.apply(lambda row: get_liwc_conj(row['liwc']), axis=1)
print('Liwc_conj wurde extrahiert!')

Liwc_conj wurde extrahiert!


In [47]:
#Für alle Tweets die Methode get_liwc_negate() wird gerufen. Die Ausgabe der Methode wird gespeichert. 
df['negate'] = df.apply(lambda row: get_liwc_negate(row['liwc']), axis=1)
print('Liwc_negate wurde extrahiert!')

Liwc_negate wurde extrahiert!


In [48]:
#Für alle Tweets die Methode get_liwc_verb() wird gerufen. Die Ausgabe der Methode wird gespeichert. 
df['verb'] = df.apply(lambda row: get_liwc_verb(row['liwc']), axis=1)
print('Liwc_verb wurde extrahiert!')

Liwc_verb wurde extrahiert!


In [49]:
#Für alle Tweets die Methode get_liwc_adj() wird gerufen. Die Ausgabe der Methode wird gespeichert. 
df['adj'] = df.apply(lambda row: get_liwc_adj(row['liwc']), axis=1)
print('Liwc_adj wurde extrahiert!')

Liwc_adj wurde extrahiert!


In [50]:
#Für alle Tweets die Methode get_liwc_compare() wird gerufen. Die Ausgabe der Methode wird gespeichert. 
df['compare'] = df.apply(lambda row: get_liwc_compare(row['liwc']), axis=1)
print('Liwc_compare wurde extrahiert!')

Liwc_compare wurde extrahiert!


In [51]:
#Für alle Tweets die Methode get_liwc_interrog() wird gerufen. Die Ausgabe der Methode wird gespeichert. 
df['interrog'] = df.apply(lambda row: get_liwc_interrog(row['liwc']), axis=1)
print('Liwc_interrog wurde extrahiert!')

Liwc_interrog wurde extrahiert!


In [52]:
#Für alle Tweets die Methode get_liwc_number() wird gerufen. Die Ausgabe der Methode wird gespeichert. 
df['number'] = df.apply(lambda row: get_liwc_number(row['liwc']), axis=1)
print('Liwc_number wurde extrahiert!')

Liwc_number wurde extrahiert!


In [53]:
#Für alle Tweets die Methode get_liwc_quant() wird gerufen. Die Ausgabe der Methode wird gespeichert. 
df['quant'] = df.apply(lambda row: get_liwc_quant(row['liwc']), axis=1)
print('Liwc_quant wurde extrahiert!')

Liwc_quant wurde extrahiert!


In [54]:
#Für alle Tweets die Methode get_liwc_see() wird gerufen. Die Ausgabe der Methode wird gespeichert. 
df['see'] = df.apply(lambda row: get_liwc_see(row['liwc']), axis=1)
print('Liwc_see wurde extrahiert!')

Liwc_see wurde extrahiert!


In [55]:
#Für alle Tweets die Methode get_liwc_hear() wird gerufen. Die Ausgabe der Methode wird gespeichert. 
df['hear'] = df.apply(lambda row: get_liwc_hear(row['liwc']), axis=1)
print('Liwc_hear wurde extrahiert!')

Liwc_hear wurde extrahiert!


In [56]:
#Für alle Tweets die Methode get_liwc_feel() wird gerufen. Die Ausgabe der Methode wird gespeichert. 
df['feel'] = df.apply(lambda row: get_liwc_feel(row['liwc']), axis=1)
print('Liwc_feel wurde extrahiert!')

Liwc_feel wurde extrahiert!


In [57]:
#Für alle Tweets die Methode get_liwc_motion() wird gerufen. Die Ausgabe der Methode wird gespeichert. 
df['motion'] = df.apply(lambda row: get_liwc_motion(row['liwc']), axis=1)
print('Liwc_motion wurde extrahiert!')

Liwc_motion wurde extrahiert!


In [58]:
#Für alle Tweets die Methode get_liwc_space() wird gerufen. Die Ausgabe der Methode wird gespeichert. 
df['space'] = df.apply(lambda row: get_liwc_space(row['liwc']), axis=1)
print('Liwc_space wurde extrahiert!')

Liwc_space wurde extrahiert!


In [59]:
#Für alle Tweets die Methode get_liwc_time() wird gerufen. Die Ausgabe der Methode wird gespeichert. 
df['time'] = df.apply(lambda row: get_liwc_time(row['liwc']), axis=1)
print('Liwc_time wurde extrahiert!')

Liwc_time wurde extrahiert!


In [60]:
#Die Resultaten speichern.
TweetsKB_Pfad =  'C:/Mo/tweetDB/tweetscov19/TweetsCov19_tsv/data_sample/TweetsKB_stichprobe_liwc_time.tsv'

with open(TweetsKB_Pfad, 'w', newline='', encoding="utf-8") as out_file:
    tsv_writer = csv.writer(out_file, delimiter="\t")
    for idx, row in df.iterrows():
        tsv_writer.writerow(row.values.tolist())